In [ ]:
import os, glob
import pandas as pd
from io import StringIO
import pyBigWig
import subprocess
import pybedtools 
pybedtools.helpers.set_tempdir('//data/private/pdutta/PSB_Data')

In [2]:
transcript_file_path = "/data/private/pdutta/PSB_Data/all_tc_input.csv"

In [3]:
df_transcript = pd.read_csv(transcript_file_path, sep=",")
df_transcript['chromosome_name'] = 'chr' + df_transcript['chromosome_name'].astype(str)
df_transcript

,Unnamed: 0,transcription_start_site,chromosome_name,transcript_start,transcript_end,strand,ensembl_gene_id,ensembl_transcript_id
0,7,22441458,chrY,22439593,22441458,-1,ENSG00000131538,ENST00000253838
1,8,6390431,chrY,6390431,6411564,1,ENSG00000129816,ENST00000250776
2,9,9774289,chrY,9753156,9774289,-1,ENSG00000129845,ENST00000250805
3,10,22144966,chrY,22144966,22146831,1,ENSG00000131548,ENST00000253848
4,11,11981507,chr18,11981507,12030877,1,ENSG00000141401,ENST00000269159
...,...,...,...,...,...,...,...,...
252697,274574,181732566,chr1,181732566,181767884,1,ENSG00000198216,ENST00000700189
252698,274575,181793668,chr1,181793668,181813262,1,ENSG00000198216,ENST00000700190
252699,274576,182329213,chr1,182278254,182329213,-1,ENSG00000228918,ENST00000702781
252700,274577,84621034,chr1,84614068,84621034,-1,ENSG00000289881,ENST00000701339


In [4]:
df_transcript[df_transcript['ensembl_transcript_id']=='ENST00000003583']

,Unnamed: 0,transcription_start_site,chromosome_name,transcript_start,transcript_end,strand,ensembl_gene_id,ensembl_transcript_id
3578,3593,24413725,chr1,24357005,24413725,-1,ENSG00000001460,ENST00000003583


## For acceptor region

In [5]:
df_acceptor = pd.read_csv("/data/private/pdutta/DNABERT_data/Splice_sites/Acceptor_raw_files/Acceptor_RAW_Combined.tsv", sep="\t")

In [6]:
df_acceptor

,Unnamed: 0,transcript_id,exon_id,transcript_type,strand,coordinates,sequence,chr_name,acceptor_40BP
0,0,ENST00000003583.12,ENSE00003692979.1,protein_coding,-,24401457,TCAACAGTGACTTCTTACGCAGTTCCTACTGCAGGCAGCACCAGCA...,chr1,GTGAAATGATCTCTACAAATGCTGTGTTTTGCTGTTTCAGGTGCTT...
1,1,ENST00000003583.12,ENSE00003638911.1,protein_coding,-,24384004,TTCCAGCCCCACCACTTGGTAACTGAGACGAAGATCGGGCAAGTCA...,chr1,TTGAAGTATCTCGATGACACCACCTTCTCTTCCCTTTCAGAATGAT...
2,2,ENST00000003583.12,ENSE00003688858.1,protein_coding,-,24379824,AAGCGAGTACAGGTCTGAACTTGCGTTTTCCAAGGGAAGATCCTCA...,chr1,TTATGTGACAATGCTGATTCCTTTGGTTGTCTCCTTACAGTGCGCC...
3,3,ENST00000003583.12,ENSE00003631141.1,protein_coding,-,24373811,AAACTCATAGACCACTTGAGCTGGGATGACAGGGAAGCCACAGCCA...,chr1,GAAAGGTACTGAGTACATTGGGTGTATTTTCTCCCCCTAGGCCTCT...
4,4,ENST00000003583.12,ENSE00003516207.1,protein_coding,-,24369840,AGCTAAGTGTGTGCTTCCCGTGCCATCTGTTCTCTGCATGGTGAGG...,chr1,AAGAGTTCCTTATTCTGTCCTAAAATTTCTTTCTCCTTAGGGCATT...
...,...,...,...,...,...,...,...,...,...
1255761,1261995,ENST00000684226.1,ENSE00003767626.1,non_stop_decay,-,13297849,AATGTCTCGCATATTGTGACTGCTCTTTACGTTAGGTTTTTTTTTA...,chrY,ACTTTGGGTTACTTTATGACTGTTGCTTTTTTCCTTCTAGCCTGCC...
1255762,1261996,ENST00000684226.1,ENSE00003761778.1,non_stop_decay,-,13260405,GTCAAATGTCAGTTATTCTCAACTTTAAATATTTTAATATTGTCAT...,chrY,TTCAGTAGTTACTCTCACAAATAGTTTTTTCCCACAATAGGTATTG...
1255763,1261997,ENST00000684226.1,ENSE00001713248.1,non_stop_decay,-,13251188,TTTCTCTGGGTGTGCTACAAAACAAAACCTAGTGTCTCAAACTCAC...,chrY,ACTGTCCAATACTTTTTGTTTATTTATTTATTTAATATAGGTGGAG...
1255764,1261998,ENST00000684226.1,ENSE00003917271.1,non_stop_decay,-,13234827,AGCTTGCACTGACCTGCAGGTGTTCCTTGATATATACATCCTGGGT...,chrY,TCCTGCAAACCCCAATGTGAACTTGTGGTGCCTTTTCCAGGCCTCC...


In [7]:
df_acceptor['start'] = df_acceptor['coordinates']-40
df_acceptor['end'] = df_acceptor['coordinates']+39

In [8]:
df_acceptor

,Unnamed: 0,transcript_id,exon_id,transcript_type,strand,coordinates,sequence,chr_name,acceptor_40BP,start,end
0,0,ENST00000003583.12,ENSE00003692979.1,protein_coding,-,24401457,TCAACAGTGACTTCTTACGCAGTTCCTACTGCAGGCAGCACCAGCA...,chr1,GTGAAATGATCTCTACAAATGCTGTGTTTTGCTGTTTCAGGTGCTT...,24401417,24401496
1,1,ENST00000003583.12,ENSE00003638911.1,protein_coding,-,24384004,TTCCAGCCCCACCACTTGGTAACTGAGACGAAGATCGGGCAAGTCA...,chr1,TTGAAGTATCTCGATGACACCACCTTCTCTTCCCTTTCAGAATGAT...,24383964,24384043
2,2,ENST00000003583.12,ENSE00003688858.1,protein_coding,-,24379824,AAGCGAGTACAGGTCTGAACTTGCGTTTTCCAAGGGAAGATCCTCA...,chr1,TTATGTGACAATGCTGATTCCTTTGGTTGTCTCCTTACAGTGCGCC...,24379784,24379863
3,3,ENST00000003583.12,ENSE00003631141.1,protein_coding,-,24373811,AAACTCATAGACCACTTGAGCTGGGATGACAGGGAAGCCACAGCCA...,chr1,GAAAGGTACTGAGTACATTGGGTGTATTTTCTCCCCCTAGGCCTCT...,24373771,24373850
4,4,ENST00000003583.12,ENSE00003516207.1,protein_coding,-,24369840,AGCTAAGTGTGTGCTTCCCGTGCCATCTGTTCTCTGCATGGTGAGG...,chr1,AAGAGTTCCTTATTCTGTCCTAAAATTTCTTTCTCCTTAGGGCATT...,24369800,24369879
...,...,...,...,...,...,...,...,...,...,...,...
1255761,1261995,ENST00000684226.1,ENSE00003767626.1,non_stop_decay,-,13297849,AATGTCTCGCATATTGTGACTGCTCTTTACGTTAGGTTTTTTTTTA...,chrY,ACTTTGGGTTACTTTATGACTGTTGCTTTTTTCCTTCTAGCCTGCC...,13297809,13297888
1255762,1261996,ENST00000684226.1,ENSE00003761778.1,non_stop_decay,-,13260405,GTCAAATGTCAGTTATTCTCAACTTTAAATATTTTAATATTGTCAT...,chrY,TTCAGTAGTTACTCTCACAAATAGTTTTTTCCCACAATAGGTATTG...,13260365,13260444
1255763,1261997,ENST00000684226.1,ENSE00001713248.1,non_stop_decay,-,13251188,TTTCTCTGGGTGTGCTACAAAACAAAACCTAGTGTCTCAAACTCAC...,chrY,ACTGTCCAATACTTTTTGTTTATTTATTTATTTAATATAGGTGGAG...,13251148,13251227
1255764,1261998,ENST00000684226.1,ENSE00003917271.1,non_stop_decay,-,13234827,AGCTTGCACTGACCTGCAGGTGTTCCTTGATATATACATCCTGGGT...,chrY,TCCTGCAAACCCCAATGTGAACTTGTGGTGCCTTTTCCAGGCCTCC...,13234787,13234866


In [9]:
df_acceptor['acceptor_40BP'].iloc[1]

'TTGAAGTATCTCGATGACACCACCTTCTCTTCCCTTTCAGAATGATATCCCAGGACCTGGGTTCTACAATGTTATTCACC'

In [10]:
df_acceptor[['transcript_id','exon_id','transcript_type','strand','chr_name', 'acceptor_40BP','start','end']].to_csv("/data/private/pdutta/DNABERT_data/Splice_sites/Acceptor_raw_files/Acceptor_80BP_bed.tsv", sep="\t", index= False)

In [11]:
df_acceptor[df_acceptor['coordinates']==22441458]

,Unnamed: 0,transcript_id,exon_id,transcript_type,strand,coordinates,sequence,chr_name,acceptor_40BP,start,end


# Not required

In [29]:
accep_trans_list = list(df_acceptor['transcript_id'].str.split('.', expand=True)[0])
transcript_list = list(df_transcript['ensembl_transcript_id'])

In [30]:
len(transcript_list)

252702

In [31]:
common_list = list(set(accep_trans_list).intersection(set(transcript_list)))

In [32]:
len(common_list)

208682

In [33]:
df_acceptor['transcript_id'] =df_acceptor['transcript_id'].str.split('.', expand=True)[0]

In [34]:
df_acceptor

,Unnamed: 0,transcript_id,exon_id,transcript_type,strand,coordinates,sequence,chr_name,acceptor_40BP,start,end
0,0,ENST00000003583,ENSE00003692979.1,protein_coding,-,24401457,TCAACAGTGACTTCTTACGCAGTTCCTACTGCAGGCAGCACCAGCA...,chr1,GTGAAATGATCTCTACAAATGCTGTGTTTTGCTGTTTCAGGTGCTT...,24401417,24401496
1,1,ENST00000003583,ENSE00003638911.1,protein_coding,-,24384004,TTCCAGCCCCACCACTTGGTAACTGAGACGAAGATCGGGCAAGTCA...,chr1,TTGAAGTATCTCGATGACACCACCTTCTCTTCCCTTTCAGAATGAT...,24383964,24384043
2,2,ENST00000003583,ENSE00003688858.1,protein_coding,-,24379824,AAGCGAGTACAGGTCTGAACTTGCGTTTTCCAAGGGAAGATCCTCA...,chr1,TTATGTGACAATGCTGATTCCTTTGGTTGTCTCCTTACAGTGCGCC...,24379784,24379863
3,3,ENST00000003583,ENSE00003631141.1,protein_coding,-,24373811,AAACTCATAGACCACTTGAGCTGGGATGACAGGGAAGCCACAGCCA...,chr1,GAAAGGTACTGAGTACATTGGGTGTATTTTCTCCCCCTAGGCCTCT...,24373771,24373850
4,4,ENST00000003583,ENSE00003516207.1,protein_coding,-,24369840,AGCTAAGTGTGTGCTTCCCGTGCCATCTGTTCTCTGCATGGTGAGG...,chr1,AAGAGTTCCTTATTCTGTCCTAAAATTTCTTTCTCCTTAGGGCATT...,24369800,24369879
...,...,...,...,...,...,...,...,...,...,...,...
1255761,1261995,ENST00000684226,ENSE00003767626.1,non_stop_decay,-,13297849,AATGTCTCGCATATTGTGACTGCTCTTTACGTTAGGTTTTTTTTTA...,chrY,ACTTTGGGTTACTTTATGACTGTTGCTTTTTTCCTTCTAGCCTGCC...,13297809,13297888
1255762,1261996,ENST00000684226,ENSE00003761778.1,non_stop_decay,-,13260405,GTCAAATGTCAGTTATTCTCAACTTTAAATATTTTAATATTGTCAT...,chrY,TTCAGTAGTTACTCTCACAAATAGTTTTTTCCCACAATAGGTATTG...,13260365,13260444
1255763,1261997,ENST00000684226,ENSE00001713248.1,non_stop_decay,-,13251188,TTTCTCTGGGTGTGCTACAAAACAAAACCTAGTGTCTCAAACTCAC...,chrY,ACTGTCCAATACTTTTTGTTTATTTATTTATTTAATATAGGTGGAG...,13251148,13251227
1255764,1261998,ENST00000684226,ENSE00003917271.1,non_stop_decay,-,13234827,AGCTTGCACTGACCTGCAGGTGTTCCTTGATATATACATCCTGGGT...,chrY,TCCTGCAAACCCCAATGTGAACTTGTGGTGCCTTTTCCAGGCCTCC...,13234787,13234866


In [35]:
df_new = df_acceptor[df_acceptor['transcript_id'].isin(common_list)][['transcript_id','exon_id','chr_name','strand','acceptor_40BP','start','end']]

In [36]:
df_new

,transcript_id,exon_id,chr_name,strand,acceptor_40BP,start,end
0,ENST00000003583,ENSE00003692979.1,chr1,-,GTGAAATGATCTCTACAAATGCTGTGTTTTGCTGTTTCAGGTGCTT...,24401417,24401496
1,ENST00000003583,ENSE00003638911.1,chr1,-,TTGAAGTATCTCGATGACACCACCTTCTCTTCCCTTTCAGAATGAT...,24383964,24384043
2,ENST00000003583,ENSE00003688858.1,chr1,-,TTATGTGACAATGCTGATTCCTTTGGTTGTCTCCTTACAGTGCGCC...,24379784,24379863
3,ENST00000003583,ENSE00003631141.1,chr1,-,GAAAGGTACTGAGTACATTGGGTGTATTTTCTCCCCCTAGGCCTCT...,24373771,24373850
4,ENST00000003583,ENSE00003516207.1,chr1,-,AAGAGTTCCTTATTCTGTCCTAAAATTTCTTTCTCCTTAGGGCATT...,24369800,24369879
...,...,...,...,...,...,...,...
1255761,ENST00000684226,ENSE00003767626.1,chrY,-,ACTTTGGGTTACTTTATGACTGTTGCTTTTTTCCTTCTAGCCTGCC...,13297809,13297888
1255762,ENST00000684226,ENSE00003761778.1,chrY,-,TTCAGTAGTTACTCTCACAAATAGTTTTTTCCCACAATAGGTATTG...,13260365,13260444
1255763,ENST00000684226,ENSE00001713248.1,chrY,-,ACTGTCCAATACTTTTTGTTTATTTATTTATTTAATATAGGTGGAG...,13251148,13251227
1255764,ENST00000684226,ENSE00003917271.1,chrY,-,TCCTGCAAACCCCAATGTGAACTTGTGGTGCCTTTTCCAGGCCTCC...,13234787,13234866


In [ ]:
24357005	24413725

In [39]:
df_new[df_new['transcript_id']=='ENST00000003583']

,transcript_id,exon_id,chr_name,strand,acceptor_40BP,start,end
0,ENST00000003583,ENSE00003692979.1,chr1,-,GTGAAATGATCTCTACAAATGCTGTGTTTTGCTGTTTCAGGTGCTT...,24401417,24401496
1,ENST00000003583,ENSE00003638911.1,chr1,-,TTGAAGTATCTCGATGACACCACCTTCTCTTCCCTTTCAGAATGAT...,24383964,24384043
2,ENST00000003583,ENSE00003688858.1,chr1,-,TTATGTGACAATGCTGATTCCTTTGGTTGTCTCCTTACAGTGCGCC...,24379784,24379863
3,ENST00000003583,ENSE00003631141.1,chr1,-,GAAAGGTACTGAGTACATTGGGTGTATTTTCTCCCCCTAGGCCTCT...,24373771,24373850
4,ENST00000003583,ENSE00003516207.1,chr1,-,AAGAGTTCCTTATTCTGTCCTAAAATTTCTTTCTCCTTAGGGCATT...,24369800,24369879
5,ENST00000003583,ENSE00003539806.1,chr1,-,ACTGCCTTAGTGACATCTTCCCGTTTTATCTTTCCCAAAGGAAAAA...,24361002,24361081
6,ENST00000003583,ENSE00003692602.1,chr1,-,TTTTCTCTCTTAATGCCTCCGCCTCTCTGTCCCCTCACAGCTACGT...,24358580,24358659


In [ ]:
# Till this

In [13]:
str_io_acceptor = StringIO()
df_acceptor[['chr_name','start','end','transcript_id','strand']].to_csv(str_io_acceptor, sep='\t', index=False, header=False)
str_io_acceptor.seek(0)
acceptor_bed = pybedtools.BedTool(str_io_acceptor.read(), from_string=True)

In [14]:
str_io_transcript = StringIO()
df_transcript[['chromosome_name','transcript_start','transcript_end','ensembl_transcript_id','strand']].to_csv(str_io_transcript, sep='\t', index=False, header=False)
str_io_transcript.seek(0)
transcript_bed = pybedtools.BedTool(str_io_transcript.read(), from_string=True)

In [15]:
# intersect the files
intersect_bed = acceptor_bed.intersect(transcript_bed, wa=True, wb=True)

In [16]:
for i, interval in enumerate(intersect_bed):
    if i < 5:
        print(interval)
    else:
        break

chr1	24401417	24401496	ENST00000003583.12	-	chr1	24401079	24410779	ENST00000475760	-1

chr1	24401417	24401496	ENST00000003583.12	-	chr1	24391562	24412027	ENST00000483528	-1

chr1	24401417	24401496	ENST00000003583.12	-	chr1	24391258	24416153	ENST00000498488	-1

chr1	24401417	24401496	ENST00000003583.12	-	chr1	24357005	24413725	ENST00000003583	-1

chr1	24401417	24401496	ENST00000003583.12	-	chr1	24356999	24413782	ENST00000337248	-1



In [18]:
intersect_df = intersect_bed.to_dataframe()

In [19]:
intersect_df.head()

,chrom,start,end,name,score,strand,thickStart,thickEnd,itemRgb,blockCount
0,chr1,24401417,24401496,ENST00000003583.12,-,chr1,24401079,24410779,ENST00000475760,-1
1,chr1,24401417,24401496,ENST00000003583.12,-,chr1,24391562,24412027,ENST00000483528,-1
2,chr1,24401417,24401496,ENST00000003583.12,-,chr1,24391258,24416153,ENST00000498488,-1
3,chr1,24401417,24401496,ENST00000003583.12,-,chr1,24357005,24413725,ENST00000003583,-1
4,chr1,24401417,24401496,ENST00000003583.12,-,chr1,24356999,24413782,ENST00000337248,-1


In [21]:
output_df.head()

,chromosome name,start,end,Acceptor_TranscriptID,strand,Intersect_Type,Ensembl_Transcript_ID
0,chr1,24401417,24401496,ENST00000003583.12,-,Partially,chr1
1,chr1,24401417,24401496,ENST00000003583.12,-,Partially,chr1
2,chr1,24401417,24401496,ENST00000003583.12,-,Partially,chr1
3,chr1,24401417,24401496,ENST00000003583.12,-,Fully,chr1
4,chr1,24401417,24401496,ENST00000003583.12,-,Partially,chr1


In [22]:
intersect_df.to_csv("/data/private/pdutta/DNABERT_data/Splice_sites/Acceptor_raw_files/Acceptor_Transcript_intersect.tsv", sep='\t', index=False)
output_df.to_csv("/data/private/pdutta/DNABERT_data/Splice_sites/Acceptor_raw_files/PART_Acceptor_Transcript_intersect_details.tsv", sep='\t', index=False)

In [50]:
chr_wise_DBSNP_files = glob.glob('/data/projects/VCF_files/DBSNP/ChrWise_Data/*.csv')

In [52]:
for dbsnp_file in chr_wise_DBSNP_files:
    # Read the csv file into a DataFrame
    print(dbsnp_file)
    df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])
    df = df.iloc[1:]
    print(df.head())
    
    # Convert the DataFrame to a BedTool
    dbsnp = pybedtools.BedTool.from_dataframe(df)
    
    # Intersect the BedTool with the transcript file
    intersected = intersect_bed.intersect(dbsnp, wa=True, wb=True)
    
    # Save the intersected BedTool to a bed file
    output_file = "/data/private/pdutta/PSB_Data/Acceptor/Chrwise/intersected_DBSNP_transcript_" + dbsnp_file.split('/')[-1].replace('.csv', '.tsv')
    intersected.saveas(output_file)

/data/projects/VCF_files/DBSNP/ChrWise_Data/chr7_data.csv


//data/private/pdutta/PSB_Data/ipykernel_1445450/2573811785.py:4: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])


      0      1      2             3  4  5   6
1  chr7  10001  10002  rs1583156258  T  1  C,
2  chr7  10002  10003  rs1583156262  A  1  G,
3  chr7  10003  10004  rs1583156265  A  1  C,
4  chr7  10004  10005  rs1583156268  C  1  A,
5  chr7  10006  10007  rs1781584841  C  1  T,
/data/projects/VCF_files/DBSNP/ChrWise_Data/chr13_data.csv


//data/private/pdutta/PSB_Data/ipykernel_1445450/2573811785.py:4: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])


       0         1         2             3  4  5   6
1  chr13  16000025  16000026  rs1175737345  T  1  A,
2  chr13  16000063  16000064  rs1957610123  G  1  A,
3  chr13  16000079  16000080  rs1957610132  G  1  T,
4  chr13  16000081  16000082  rs1957610136  A  1  C,
5  chr13  16000100  16000101  rs1957610144  T  1  C,
/data/projects/VCF_files/DBSNP/ChrWise_Data/chrY_data.csv


...

      0      1      2             3   4  5       6
1  chr3  10008  10009  rs1339898020   A  3  C,G,T,
2  chr3  10009  10010  rs1695495101   A  1      C,
3  chr3  10010  10011  rs1695495136   C  1      A,
4  chr3  10014  10015  rs1206436362   A  1      C,
5  chr3  10014  10016  rs1695495167  AA  1    AAA,
/data/projects/VCF_files/DBSNP/ChrWise_Data/chr17_data.csv


//data/private/pdutta/PSB_Data/ipykernel_1445450/2573811785.py:4: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])


       0      1      2             3  4  5   6
1  chr17  60054  60055  rs2039333033  G  1  A,
2  chr17  60086  60087  rs2039333047  C  1  T,
3  chr17  60102  60103  rs1399099657  G  1  A,
4  chr17  60103  60104  rs1363626035  C  1  T,
5  chr17  60149  60150  rs2039333080  C  1  A,
/data/projects/VCF_files/DBSNP/ChrWise_Data/chr2_data.csv


//data/private/pdutta/PSB_Data/ipykernel_1445450/2573811785.py:4: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])


      0      1      2             3        4  5         6
1  chr2  10006  10007  rs1572047073        C  1        A,
2  chr2  10015  10015  rs1558169263      NaN  1        G,
3  chr2  10016  10023  rs1558169385  CACCCAC  1      CAC,
4  chr2  10018  10021  rs1558169386      CCC  1       CC,
5  chr2  10019  10019  rs1558169388      NaN  2  ACA,ACG,


In [28]:
for i, interval in enumerate(bigbed_file):
    if i < 5:
        print(interval)
    else:
        break

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xeb in position 0: invalid continuation byte

In [ ]:
intersect_acceptor_DBSNP = acceptor_bed.intersect(transcript_bed, wa=True, wb=True)

In [25]:
# Convert the BigBed file to a Bed file
bed_file_from_bigbed = bigbed_file.to_bed()

AttributeError: 'BedTool' object has no attribute 'to_bed'

In [26]:
bw = pyBigWig.open('/data/projects/VCF_files/DBSNP/dbSnp155.bb')

In [17]:
# output dataframe
output_df = pd.DataFrame(columns=["chromosome name",  "start", "end","Acceptor_TranscriptID", "strand","Intersect_Type", "Ensembl_Transcript_ID"])

for i, interval in enumerate(intersect_bed):
    # extract acceptor and transcript information
    if(i%1000==0):
        print("Number of instance", i+1)
    acceptor_info = interval[:5]
    transcript_info = interval[5:]
    
    # check if it is fully or partially intersected
    intersect_type = "Fully" if acceptor_info[3] >= transcript_info[3] and acceptor_info[4] <= transcript_info[4] else "Partially"
    row_to_add = pd.Series([*acceptor_info, intersect_type, transcript_info[0]], index=output_df.columns)
    output_df = pd.concat([output_df, pd.DataFrame([row_to_add])], ignore_index=True)

Number of instance 1
Number of instance 1001
Number of instance 2001
Number of instance 3001
Number of instance 4001
Number of instance 5001
Number of instance 6001
Number of instance 7001
Number of instance 8001
Number of instance 9001
Number of instance 10001
Number of instance 11001
Number of instance 12001
Number of instance 13001
Number of instance 14001
Number of instance 15001
Number of instance 16001
Number of instance 17001
Number of instance 18001
Number of instance 19001
Number of instance 20001
Number of instance 21001
Number of instance 22001
Number of instance 23001
Number of instance 24001
Number of instance 25001
Number of instance 26001
Number of instance 27001
Number of instance 28001
Number of instance 29001
Number of instance 30001
Number of instance 31001
Number of instance 32001
Number of instance 33001
Number of instance 34001
Number of instance 35001
Number of instance 36001
Number of instance 37001
Number of instance 38001
Number of instance 39001
Number of ins

KeyboardInterrupt: 

In [ ]:
ou